In [2]:
import pandas as pd

cxr_studies = pd.read_csv('mimiccxr_files/cxr-study-list.csv.gz', compression='gzip').rename(columns = {'path':'txt_path'})
cxr_studies.head()

FileNotFoundError: [Errno 2] No such file or directory: 'mimiccxr_files/cxr-study-list.csv.gz'

In [31]:
pathdir = 'mimiccxr_files/'
from tqdm import tqdm
def get_text(row):
    fpath= pathdir+row
    with open(fpath, 'r') as file:
        text = file.read()
    return text
tqdm.pandas(desc='assigning text...')
cxr_studies['report_text'] = cxr_studies['txt_path'].progress_apply(get_text)
cxr_studies

assigning text...: 100%|██████████| 227835/227835 [01:01<00:00, 3715.38it/s]


,subject_id,study_id,txt_path,report_text
0,10000032,50414267,files/p10/p10000032/s50414267.txt,FINAL REPORT\...
1,10000032,53189527,files/p10/p10000032/s53189527.txt,FINAL REPORT\...
2,10000032,53911762,files/p10/p10000032/s53911762.txt,FINAL REPORT\...
3,10000032,56699142,files/p10/p10000032/s56699142.txt,FINAL REPORT\...
4,10000764,57375967,files/p10/p10000764/s57375967.txt,FINAL REPORT\...
...,...,...,...,...
227830,19999442,58708861,files/p19/p19999442/s58708861.txt,FINAL REPORT\...
227831,19999733,57132437,files/p19/p19999733/s57132437.txt,FINAL REPORT\...
227832,19999987,55368167,files/p19/p19999987/s55368167.txt,FINAL REPORT\...
227833,19999987,58621812,files/p19/p19999987/s58621812.txt,FINAL REPORT\...


In [32]:
cxr_split_df  = pd.read_csv('mimiccxr_jpg_and_labels/mimic-cxr-2.0.0-split.csv.gz',usecols=['study_id','split'])
cxr_studies = cxr_studies.merge(cxr_split_df, on = 'study_id')
cxr_studies=cxr_studies.drop_duplicates(subset='report_text')
cxr_studies

,subject_id,study_id,txt_path,report_text,split
0,10000032,50414267,files/p10/p10000032/s50414267.txt,FINAL REPORT\...,train
2,10000032,53189527,files/p10/p10000032/s53189527.txt,FINAL REPORT\...,train
4,10000032,53911762,files/p10/p10000032/s53911762.txt,FINAL REPORT\...,train
6,10000032,56699142,files/p10/p10000032/s56699142.txt,FINAL REPORT\...,train
7,10000764,57375967,files/p10/p10000764/s57375967.txt,FINAL REPORT\...,train
...,...,...,...,...,...
377103,19999442,58708861,files/p19/p19999442/s58708861.txt,FINAL REPORT\...,train
377104,19999733,57132437,files/p19/p19999733/s57132437.txt,FINAL REPORT\...,train
377107,19999987,55368167,files/p19/p19999987/s55368167.txt,FINAL REPORT\...,train
377108,19999987,58621812,files/p19/p19999987/s58621812.txt,FINAL REPORT\...,train


In [14]:
mimiciv_diag_key_df = pd.read_csv('mimiciv/d_icd_diagnoses.csv') #getting ICD10 only.
mimiciv_diag_key_df = mimiciv_diag_key_df[mimiciv_diag_key_df.icd_version == 10]
mimiciv_diag_key_df

,icd_code,icd_version,long_title
9995,A3682,10,Diphtheritic radiculomyelitis
10474,A3683,10,Diphtheritic polyneuritis
10525,A5052,10,Hutchinson's teeth
10616,A5053,10,Hutchinson's triad
10672,A881,10,Epidemic vertigo
...,...,...,...
86746,Z9912,10,Encounter for respirator [ventilator] dependen...
86747,Z992,10,Dependence on renal dialysis
86748,Z993,10,Dependence on wheelchair
86749,Z9981,10,Dependence on supplemental oxygen


In [18]:
#loading mimic-iv...
mimiciv_diag_df = pd.read_csv('mimiciv/diagnoses_icd.csv')
mimiciv_diag_df = mimiciv_diag_df[mimiciv_diag_df.icd_version == 10]
mimiciv_diag_df = mimiciv_diag_df.merge(mimiciv_diag_key_df.drop('icd_version',axis=1),on='icd_code')
mimiciv_diag_df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,18335503,21596415,7,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
1,10071611,27474298,9,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
2,16855850,24861829,17,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
3,15658622,20448464,14,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
4,13137719,20370714,5,O99282,10,"Endocrine, nutritional and metabolic diseases ..."


In [36]:
mimiciv_diag_df

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,18335503,21596415,7,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
1,10071611,27474298,9,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
2,16855850,24861829,17,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
3,15658622,20448464,14,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
4,13137719,20370714,5,O99282,10,"Endocrine, nutritional and metabolic diseases ..."
...,...,...,...,...,...,...
2149978,15514518,21042034,2,S59801A,10,"Other specified injuries of right elbow, initi..."
2149979,13035200,27999585,2,C8241,10,"Follicular lymphoma grade IIIb, lymph nodes of..."
2149980,17111939,23664945,1,T6101XA,10,"Ciguatera fish poisoning, accidental (unintent..."
2149981,19135001,27980505,2,S92502A,10,Displaced unspecified fracture of left lesser ...


use the subject_id to link IV and CXR,
if you want to assign an hadm_id, then join using subject_id + admittime + dischtime

In [1]:
pd.read_csv('mimiciv/admissions.csv')

NameError: name 'pd' is not defined